In [50]:
import numpy as np
from tqdm import tqdm, trange

import torch
import torch.nn as nn
from torch import optim
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, Subset, default_collate
from torchvision import transforms
from torchvision.transforms import ToTensor, v2
from torchvision import datasets

np.random.seed(0)
torch.manual_seed(0)

In [2]:
# load fashion MNIST data and transform images
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

In [3]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [4]:
def patchify(images, n_patches):
    n, c, h, w = images.shape # n=num_images, c=image dimension

    # ensure input images are squares
    assert h==w, "image must be square"

    # instantiate patches as a 3D zero tensor (** means power of)
    patches = torch.zeros(n, n_patches**2, h*w*c // n_patches**2) # (num_images, num_patches, patch dimension)
    patch_size = h //n_patches

    for index, image in enumerate(images):
        for i in range(n_patches):
            for j in range(n_patches):
                patch = image[:, i*patch_size: (i+1)*patch_size, j*patch_size: (j+1)*patch_size]
                patches[index, i*n_patches + j] = patch.flatten()
    return patches


In [5]:
def get_positional_embedding(seq_length, dim):
    pe = torch.ones(seq_length, dim)
    for i in range(seq_length):
        for j in range(dim):
            if j % 2 == 0: # even
                pe[i][j] = np.sin(i/(10000 ** (j/dim)))
            elif j % 2 == 1: # odd
                pe[i][j] = np.cos(i / (10000 ** ((j-1)/dim)))
    return pe

In [6]:
class MultiHeadSA(nn.Module):
    def __init__(self, dim, n_heads=2):
        super(MultiHeadSA, self).__init__()
        self.dim = dim
        self.n_heads = n_heads

        assert dim % n_heads == 0, f"Can't divide dimension {dim} into {n_heads} heads"

        # creating weight matrix of q, k and v
        d_head = int(dim / n_heads)
        self.q_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for x in range(self.n_heads)])
        self.k_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for x in range(self.n_heads)])
        self.v_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for x in range(self.n_heads)])
        self.d_head = d_head
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, sequences):
        # Sequences has shape (N, seq_length, token_dim)
        # We go into shape    (N, seq_length, n_heads, token_dim / n_heads)
        # And come back to    (N, seq_length, item_dim)  (through concatenation)
        result = []
        for sequence in sequences:
            seq_result = []
            for head in range(self.n_heads):
                q_mapping = self.q_mappings[head]
                k_mapping = self.k_mappings[head]
                v_mapping = self.v_mappings[head]

                seq = sequence[:, head * self.d_head: (head + 1) * self.d_head]

                q, k, v = q_mapping(seq), k_mapping(seq), v_mapping(seq)
                
                # calculate attention score
                attention = self.softmax(q @ k.T / (self.d_head ** 0.5))
                seq_result.append(attention @ v)
            result.append(torch.hstack(seq_result))

        # concat attention from each head together
        return torch.cat([torch.unsqueeze(r, dim=0) for r in result])

In [7]:
class ResidualConnection(nn.Module):
    def __init__(self, hidden_dim, n_heads, mlp_ratio=4):
        super(ResidualConnection, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_heads = n_heads

        self.norm1 = nn.LayerNorm(hidden_dim)

        # self-attention layer
        self.mhsa = MultiHeadSA(hidden_dim, n_heads)

        self.norm2 = nn.LayerNorm(hidden_dim)

        # feed forward layer
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, mlp_ratio * hidden_dim),
            nn.GELU(),
            nn.Linear(mlp_ratio * hidden_dim, hidden_dim)
        )

    def forward(self, x):
        mhsa_out = x + self.mhsa(self.norm1(x))
        ff_out = mhsa_out + self.mlp(self.norm2(mhsa_out))
        return ff_out

In [8]:
# instantiate model
class VisionTransformer(nn.Module):
    def __init__(self, chw, n_patches, hidden_dim, n_encodelayers, n_heads, output_dim):
    super(VisionTransformer, self).__init__()

    self.chw = chw
    self.n_patches = n_patches
    self.hidden_dim = hidden_dim
    self.n_encodelayers = n_encodelayers
    self.n_heads = n_heads
    self.output_dim = output_dim

    # ensure that height and width are divisble by num of patches
    assert chw[1] % n_patches == 0, "Input shape is not divisible by number of patches"
    assert chw[2] % n_patches == 0, "Input shape is not divisible by number of patches"
    self.patch_size = (chw[1]/n_patches, chw[2]/n_patches)

    # linear mapping
    self.input_dim = int(chw[0] * self.patch_size[0] * self.patch_size[1])
    self.linear_map = nn.Linear(self.input_dim, self.hidden_dim)

    # classification head
    self.class_token = nn.Parameter(torch.rand(1, self.hidden_dim))

    # positional encoding
    self.pos_embed = nn.Parameter(torch.tensor(get_positional_embedding(self.n_patches ** 2 + 1, self.hidden_dim)))
    # self.pos_embed.requires_grad = False

    # transformer encoder
    self.encoder_layers = nn.ModuleList([ResidualConnection(hidden_dim, n_heads) for x in range(n_encodelayers)])

    # extract classification token
    self.mlp = nn.Sequential(
        nn.Linear(self.hidden_dim, output_dim),
        nn.Softmax(dim=-1)
    )

    def forward(self, input_image):
    patches = patchify(input_image, self.n_patches)
    tokens = self.linear_map(patches)

    # add classification head to each token (learnable embedding)
    tokens_with_class = torch.stack([torch.vstack((self.class_token, tokens[i])) for i in range(len(tokens))])

    # add positional embedding
    pos_embed = self.pos_embed.repeat(tokens_with_class.shape[0], 1, 1)
    pos_embed_token = tokens_with_class + pos_embed

    # transformer encoding layers
    for layer in self.encoder_layers:
        output = layer(pos_embed_token)

    # extract classification token
    output = output[:, 0]
    pred = self.mlp(output)

    return pred # Map to output dimension, output category distribution

In [9]:
# train and test loop
def train_loop(train_loader, model, loss_fn, optimizer, epoch):
    train_loss = 0.0
    correct, total = 0, 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1} in training", leave=False):
        x, y = batch
        output = model(x)
        loss = loss_fn(output, y)

        train_loss += (loss.item() / len(train_loader))
        correct += torch.sum(torch.argmax(output, dim=1) == y).item()
        total += len(x)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} loss: {train_loss:.2f}")
    print(f"Validation accuracy: {correct / total * 100:.2f}%")

def test_loop(test_loader, model, loss_fn):
    with torch.no_grad():
        correct, total = 0, 0
        test_loss = 0.0
        for batch in tqdm(test_loader, desc="Testing"):
            x, y = batch
            output = model(x)
            loss = loss_fn(output, y)
            test_loss += (loss.item() / len(test_loader))

            correct += torch.sum(torch.argmax(output, dim=1) == y).item()
            total += len(x)
        test_acc = correct / total * 100
        print(f"Test loss: {test_loss:.2f}")
        print(f"Test accuracy: {test_acc:.2f}%")
    return test_acc, test_loss

In [10]:
batch_size = 512

comparing different hyperparameters using subset

In [11]:
# obtain subset of data
train_small = Subset(training_data, range(int(0.6*len(training_data))))
test_small = Subset(test_data, range(int(0.6*len(test_data))))

# load data
train_dataloader = DataLoader(train_small, shuffle=True, batch_size = batch_size)
test_dataloader = DataLoader(test_small, shuffle=True, batch_size = batch_size)

### Base Transformer 

In [21]:
chw = (1, 64, 64) # image dimensions
hidden_dim = 16 # number of features in each patch's representation
n_encodelayers = 2
n_heads = 2 # no of attention heads
output_dim = 10 # Fashion MNIST has 10 classes

n_patches = 16

learning_rate = 0.005
num_epochs = 20

# instantiate model
model = VisionTransformer(chw, n_patches, hidden_dim, n_encodelayers, n_heads, output_dim)

# instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train model with training data
for epoch in range(num_epochs):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch)

/tmp/ipykernel_57930/1295966561.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embed = nn.Parameter(torch.tensor(get_positional_embedding(self.n_patches ** 2 + 1, self.hidden_dim)))


Epoch 1 loss: 2.05
Test accuracy: 41.10%


Epoch 2 loss: 1.83
Test accuracy: 63.82%


Epoch 3 loss: 1.75
Test accuracy: 71.05%


Epoch 4 loss: 1.72
Test accuracy: 74.04%


...

Epoch 16 loss: 1.66
Test accuracy: 79.65%


Epoch 17 loss: 1.67
Test accuracy: 79.47%


Epoch 18 loss: 1.66
Test accuracy: 79.90%


Epoch 19 loss: 1.66
Test accuracy: 80.14%


Epoch 20 loss: 1.66
Test accuracy: 80.03%


In [22]:
# test model to get test loss and accuracy
test_accuracy, test_loss = test_loop(test_dataloader, model, loss_fn)

Testing:   5%|▌         | 1/20 [00:02<00:55,  2.91s/it]

tensor([[2.0435e-21, 2.1317e-12, 1.4870e-31,  ..., 1.0000e+00, 1.6927e-23,
         5.1711e-10],
        [0.0000e+00, 1.0771e-10, 2.0171e-31,  ..., 2.5845e-16, 3.9119e-27,
         1.0000e+00],
        [0.0000e+00, 1.8282e-12, 4.1712e-33,  ..., 1.2513e-14, 2.0280e-23,
         1.0000e+00],
        ...,
        [5.3050e-17, 1.0000e+00, 6.6771e-23,  ..., 8.5728e-24, 0.0000e+00,
         1.4401e-28],
        [5.3267e-15, 7.7979e-16, 3.8715e-19,  ..., 6.9117e-13, 5.8055e-22,
         4.4612e-17],
        [1.0000e+00, 5.0422e-29, 4.9498e-16,  ..., 1.2395e-35, 0.0000e+00,
         0.0000e+00]])


Testing:  10%|█         | 2/20 [00:05<00:52,  2.90s/it]

tensor([[1.3707e-30, 2.3906e-39, 8.8082e-17,  ..., 3.2541e-28, 9.6677e-01,
         2.9584e-22],
        [1.4013e-44, 0.0000e+00, 1.1356e-16,  ..., 0.0000e+00, 1.4348e-13,
         5.3122e-35],
        [1.4557e-13, 2.2105e-38, 8.5774e-17,  ..., 3.9195e-26, 8.8302e-09,
         6.0864e-29],
        ...,
        [5.4569e-21, 9.9371e-09, 4.3037e-29,  ..., 1.0000e+00, 7.6246e-27,
         1.6474e-08],
        [0.0000e+00, 2.3401e-11, 1.3416e-18,  ..., 1.0270e-33, 8.5072e-24,
         1.0000e+00],
        [1.7002e-20, 8.4213e-08, 4.5935e-28,  ..., 1.0000e+00, 3.7659e-24,
         7.1156e-09]])


Testing:  15%|█▌        | 3/20 [00:08<00:49,  2.89s/it]

...

tensor([[3.1995e-22, 2.0208e-18, 7.2063e-23,  ..., 1.3242e-14, 4.1829e-20,
         2.0883e-15],
        [4.1739e-34, 0.0000e+00, 5.0112e-15,  ..., 2.1019e-44, 2.6008e-08,
         1.5310e-36],
        [1.4455e-24, 1.9883e-26, 7.7545e-02,  ..., 1.4014e-31, 1.6595e-15,
         4.2745e-20],
        ...,
        [6.0612e-10, 5.0015e-24, 7.3266e-05,  ..., 3.8306e-24, 4.5550e-08,
         3.9050e-20],
        [4.1859e-23, 1.3386e-08, 3.1172e-30,  ..., 1.0000e+00, 6.0842e-26,
         3.5808e-08],
        [1.3277e-19, 1.4534e-08, 6.6612e-27,  ..., 1.0000e+00, 1.3615e-22,
         4.7098e-08]])


Testing:  95%|█████████▌| 19/20 [00:55<00:02,  2.94s/it]

tensor([[6.2147e-08, 1.0214e-11, 9.0860e-15,  ..., 1.8026e-20, 2.8269e-18,
         5.8135e-21],
        [1.8656e-08, 4.9570e-14, 1.6210e-15,  ..., 2.1437e-21, 1.8076e-16,
         9.1676e-22],
        [0.0000e+00, 2.8026e-45, 5.9373e-09,  ..., 0.0000e+00, 4.3290e-15,
         2.0861e-30],
        ...,
        [8.4043e-16, 6.3266e-20, 4.7588e-24,  ..., 1.4945e-12, 1.0292e-16,
         3.5427e-18],
        [0.0000e+00, 1.3250e-10, 4.7199e-22,  ..., 3.0691e-22, 5.2090e-24,
         1.0000e+00],
        [9.8756e-01, 2.3234e-06, 7.5138e-10,  ..., 1.5779e-27, 1.4013e-45,
         3.0489e-31]])


Testing: 100%|██████████| 20/20 [00:56<00:00,  2.83s/it]

tensor([[6.0768e-20, 3.2085e-09, 3.4560e-28,  ..., 1.0000e+00, 1.1961e-23,
         2.6276e-09],
        [1.4013e-45, 1.3018e-42, 4.2509e-28,  ..., 5.0652e-37, 1.0000e+00,
         5.7321e-24],
        [4.2019e-23, 9.5292e-22, 1.5577e-27,  ..., 3.7775e-14, 6.8352e-21,
         1.5021e-17],
        ...,
        [5.1487e-22, 1.6893e-14, 1.7729e-29,  ..., 9.5003e-04, 4.3625e-15,
         1.1604e-09],
        [9.9906e-01, 2.1967e-11, 9.3635e-04,  ..., 1.4067e-27, 2.5713e-37,
         5.7506e-29],
        [5.0045e-06, 3.4353e-09, 5.8475e-15,  ..., 6.4304e-19, 9.9453e-22,
         1.8569e-21]])
Test loss: 1.66
Test accuracy: 79.89%


### Changing lr (0.005 -> 0.001) 

In [12]:
from pathlib import Path

chw = (1, 64, 64) # image dimensions
hidden_dim = 16 # number of features in each patch's representation
n_encodelayers = 2
n_heads = 2 # no of attention heads
output_dim = 10 # Fashion MNIST has 10 classes

n_patches = 16

learning_rate = 0.001
num_epochs = 10

# instantiate model
if Path('./model_checkpoint_lr_001.pt').exists():
    model = torch.load('./model_checkpoint_lr_001.pt')
else:
    model = VisionTransformer(chw, n_patches, hidden_dim, n_encodelayers, n_heads, output_dim)

# instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train model with training data
for epoch in range(num_epochs):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch)
    torch.save(model, './model_checkpoint_lr_001.pt')
    
# test model to get test loss and accuracy
test_accuracy, test_loss = test_loop(test_dataloader, model, loss_fn)

C:\Users\Joey Quah\AppData\Local\Temp\ipykernel_17792\1295966561.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embed = nn.Parameter(torch.tensor(get_positional_embedding(self.n_patches ** 2 + 1, self.hidden_dim)))


Epoch 1 loss: 2.24
Validation accuracy: 19.81%


Epoch 2 loss: 2.03
Validation accuracy: 44.20%


Epoch 3 loss: 1.88
Validation accuracy: 59.28%


Epoch 4 loss: 1.84
Validation accuracy: 62.37%


...

tensor([[3.8951e-17, 2.0758e-09, 1.6645e-15,  ..., 1.8770e-10, 6.0904e-09,
         1.3383e-17],
        [1.8877e-09, 4.7029e-03, 7.4102e-21,  ..., 1.0572e-06, 2.3500e-05,
         9.5350e-10],
        [2.2787e-01, 6.3131e-01, 3.8949e-15,  ..., 1.9494e-13, 2.5208e-12,
         1.0177e-14],
        ...,
        [4.4046e-07, 7.7782e-11, 7.2447e-11,  ..., 3.2908e-18, 8.3925e-05,
         1.6324e-12],
        [4.8589e-26, 9.0374e-16, 9.7435e-25,  ..., 9.9920e-01, 9.0216e-06,
         7.9500e-04],
        [1.2159e-23, 5.6290e-13, 3.7588e-24,  ..., 1.0000e+00, 6.1440e-07,
         1.3053e-06]])


Testing:  95%|█████████████████████████████████████████████████████████████████████████████████▋    | 19/20 [01:15<00:03,  3.98s/it]

tensor([[9.9782e-01, 7.7541e-10, 4.7675e-06,  ..., 3.9169e-18, 1.6997e-11,
         1.5494e-13],
        [5.1115e-24, 1.2668e-12, 7.4223e-24,  ..., 9.9999e-01, 2.2842e-06,
         2.8212e-07],
        [3.3715e-12, 1.5133e-10, 2.5583e-07,  ..., 2.5291e-14, 3.1895e-05,
         5.6633e-11],
        ...,
        [9.9862e-01, 3.5178e-08, 6.8913e-11,  ..., 2.0278e-19, 2.1201e-13,
         1.5163e-16],
        [4.2150e-17, 4.2723e-06, 3.4939e-17,  ..., 3.1666e-05, 1.0256e-07,
         2.9284e-15],
        [6.2456e-01, 5.4346e-05, 1.6254e-13,  ..., 9.8031e-17, 2.1236e-10,
         2.5699e-14]])


Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████| 20/20 [01:18<00:00,  3.90s/it]

tensor([[9.0283e-06, 2.4838e-03, 4.0479e-22,  ..., 2.1293e-14, 3.9811e-12,
         5.1946e-15],
        [4.3431e-07, 2.2653e-14, 2.6869e-02,  ..., 1.5217e-13, 8.5482e-01,
         1.2065e-04],
        [3.3978e-21, 7.3561e-10, 1.6277e-21,  ..., 9.9971e-01, 1.1219e-06,
         1.5730e-08],
        ...,
        [7.9400e-08, 2.4613e-13, 9.9982e-01,  ..., 1.2174e-18, 2.7867e-08,
         1.5214e-13],
        [2.7211e-22, 2.0459e-11, 3.7749e-23,  ..., 1.0000e+00, 5.4508e-07,
         1.6266e-07],
        [7.4700e-03, 4.8557e-05, 3.8046e-13,  ..., 2.9796e-18, 3.9417e-08,
         4.5646e-16]])
Test loss: 1.68
Test accuracy: 78.55%


### Changing lr (0.005 -> 0.01)

In [ ]:
from pathlib import Path

chw = (1, 64, 64) # image dimensions
hidden_dim = 16 # number of features in each patch's representation
n_encodelayers = 2
n_heads = 2 # no of attention heads
output_dim = 10 # Fashion MNIST has 10 classes

n_patches = 16

learning_rate = 0.01
num_epochs = 20

# instantiate model
model = VisionTransformer(chw, n_patches, hidden_dim, n_encodelayers, n_heads, output_dim)

# instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train model with training data
for epoch in range(num_epochs):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch)
    torch.save(model, './model_checkpoint_lr_01.pt')
    
# test model to get test loss and accuracy
test_accuracy, test_loss = test_loop(test_dataloader, model, loss_fn)

### Base model but on smaller train set (to save on computation time) 

In [18]:
chw = (1, 64, 64) # image dimensions
hidden_dim = 16 # number of features in each patch's representation
n_encodelayers = 2
n_heads = 2 # no of attention heads
output_dim = 10 # Fashion MNIST has 10 classes

n_patches = 16

learning_rate = 0.01
num_epochs = 10

# instantiate model
model = VisionTransformer(chw, n_patches, hidden_dim, n_encodelayers, n_heads, output_dim)

# instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train model with training data
for epoch in range(num_epochs):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch)

# test model to get test loss and accuracy
test_accuracy, test_loss = test_loop(test_dataloader, model, loss_fn)

C:\Users\Joey Quah\AppData\Local\Temp\ipykernel_15160\1295966561.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embed = nn.Parameter(torch.tensor(get_positional_embedding(self.n_patches ** 2 + 1, self.hidden_dim)))


Epoch 1 loss: 2.11
Validation accuracy: 33.79%


Epoch 2 loss: 1.88
Validation accuracy: 57.66%


Epoch 3 loss: 1.83
Validation accuracy: 62.44%


Epoch 4 loss: 1.81
Validation accuracy: 64.91%


...

Epoch 8 loss: 1.79
Validation accuracy: 67.39%


Epoch 9 loss: 1.79
Validation accuracy: 67.25%


Epoch 10 loss: 1.78
Validation accuracy: 68.08%


Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:55<00:00,  4.60s/it]

Test loss: 1.78
Test accuracy: 67.82%


### Changing hidden_dim (8, 32 vs 16 base)

In [16]:
chw = (1, 64, 64) # image dimensions
n_encodelayers = 2
n_heads = 2 # no of attention heads
output_dim = 10 # Fashion MNIST has 10 classes
n_patches = 16

# parameter to test
hidden_dims = [8, 32] # number of features in each patch's representation

batch_size = 512
learning_rate = 0.01
num_epochs = 10

test_acc_list = []
for hidden_dim in hidden_dims:
    # instantiate model
    model = VisionTransformer(chw, n_patches, hidden_dim, n_encodelayers, n_heads, output_dim)
    
    # instantiate loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # train model with training data
    for epoch in range(num_epochs):
        train_loop(train_dataloader, model, loss_fn, optimizer, epoch)
        torch.save(model, f'./saved_model__hidden_dim{hidden_dim}.pt')
        
#     # test model to get test loss and accuracy
#     test_accuracy, test_loss = test_loop(test_dataloader, model, loss_fn)
    
# print(f'Test accuracy for hidden_dim 8: {test_acc_list[0]}')
# print(f'Test accuracy for hidden_dim 32: {test_acc_list[1]}')

C:\Users\Joey Quah\AppData\Local\Temp\ipykernel_17792\1295966561.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embed = nn.Parameter(torch.tensor(get_positional_embedding(self.n_patches ** 2 + 1, self.hidden_dim)))


Epoch 1 loss: 2.19
Validation accuracy: 25.73%


Epoch 2 loss: 1.92
Validation accuracy: 54.47%


Epoch 3 loss: 1.83
Validation accuracy: 62.87%


Epoch 4 loss: 1.81
Validation accuracy: 65.21%


...

Testing:  92%|██████████████████████████████████████████████████████████████████████████████▊       | 11/12 [00:47<00:04,  4.34s/it]

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.6735e-40, 6.0852e-18,
         2.3239e-26],
        [0.0000e+00, 1.6804e-31, 2.0287e-39,  ..., 9.9764e-01, 1.4628e-41,
         5.4505e-09],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.7174e-32, 1.3627e-09,
         2.4162e-14],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.8419e-32, 1.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])


Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:51<00:00,  4.27s/it]

tensor([[1.9716e-36, 1.6368e-27, 9.0908e-41,  ..., 0.0000e+00, 0.0000e+00,
         2.4447e-41],
        [2.7606e-35, 1.1342e-30, 3.0168e-30,  ..., 2.7003e-42, 3.2661e-28,
         5.2408e-29],
        [9.5002e-05, 1.6139e-25, 1.6090e-16,  ..., 7.6634e-41, 3.4091e-27,
         1.2612e-44],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.8060e-19, 2.2400e-36,
         1.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 5.5351e-24, 1.0000e+00,
         9.2594e-13],
        [7.5068e-36, 1.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])
Test loss: 1.77
Test accuracy: 69.32%


IndexError: list index out of range

In [22]:
test_acc_list = []
for hidden_dim in hidden_dims:
    model = torch.load(f'./saved_model__hidden_dim{hidden_dim}.pt')
    # test model to get test loss and accuracy
    test_accuracy, test_loss = test_loop(test_dataloader, model, loss_fn)
    test_acc_list.append(test_accuracy)
    
print(f'Test accuracy for hidden_dim 8: {test_acc_list[0]:.2f}%')
print(f'Test accuracy for hidden_dim 32: {test_acc_list[1]:.2f}%')

Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:49<00:00,  4.16s/it]


Test loss: 1.71
Test accuracy: 74.70%


Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:49<00:00,  4.16s/it]

Test loss: 1.77
Test accuracy: 69.32%
Test accuracy for hidden_dim 8: 74.70%
Test accuracy for hidden_dim 32: 69.32%


### Changing n_encodelayer (4, 8 vs 2 base)

In [12]:
chw = (1, 64, 64) # image dimensions
n_heads = 2 # no of attention heads
output_dim = 10 # Fashion MNIST has 10 classes
n_patches = 16
hidden_dim = 16 # number of features in each patch's representation

# parameter to test
n_encodelayer = [4, 8]

batch_size = 512
learning_rate = 0.01
num_epochs = 10

test_acc_list = []
for n_encodelayers in n_encodelayer:
    # instantiate model
    model = VisionTransformer(chw, n_patches, hidden_dim, n_encodelayers, n_heads, output_dim)
    
    # instantiate loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # train model with training data
    for epoch in range(num_epochs):
        train_loop(train_dataloader, model, loss_fn, optimizer, epoch)
        torch.save(model, f'./saved_model__enclayer{n_encodelayers}.pt')

    # test model to get test loss and accuracy
    test_accuracy, test_loss = test_loop(test_dataloader, model, loss_fn)
    test_acc_list.append(test_accuracy)
    
print(f'Test accuracy for n_encodelayer 4: {test_acc_list[0]:.2f}%')
print(f'Test accuracy for n_encodelayer 8: {test_acc_list[1]:.2f}%')

C:\Users\Joey Quah\AppData\Local\Temp\ipykernel_12380\1295966561.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embed = nn.Parameter(torch.tensor(get_positional_embedding(self.n_patches ** 2 + 1, self.hidden_dim)))


Epoch 1 loss: 2.11
Validation accuracy: 33.75%


Epoch 2 loss: 1.89
Validation accuracy: 56.81%


Epoch 3 loss: 1.83
Validation accuracy: 63.61%


Epoch 4 loss: 1.81
Validation accuracy: 64.75%


...

Epoch 8 loss: 1.73
Validation accuracy: 72.68%


Epoch 9 loss: 1.73
Validation accuracy: 72.57%


Epoch 10 loss: 1.73
Validation accuracy: 73.06%


Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████| 12/12 [01:25<00:00,  7.16s/it]

Test loss: 1.72
Test accuracy: 73.75%
Test accuracy for n_encodelayer 4: 70.18%
Test accuracy for n_encodelayer 8: 73.75%


### Changing n_heads (4, 8 vs 2 base)

In [17]:
chw = (1, 64, 64) # image dimensions
output_dim = 10 # Fashion MNIST has 10 classes
n_patches = 16
hidden_dim = 16 # number of features in each patch's representation
n_encodelayers = 2

# parameter to test
n_head = [4, 8] # no of attention heads

batch_size = 512
learning_rate = 0.01
num_epochs = 10

test_acc_list = []
for n_heads in n_head:
    # instantiate model
    model = VisionTransformer(chw, n_patches, hidden_dim, n_encodelayers, n_heads, output_dim)
    
    # instantiate loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # train model with training data
    for epoch in range(num_epochs):
        train_loop(train_dataloader, model, loss_fn, optimizer, epoch)
        torch.save(model, f'./saved_model__n_head{n_heads}.pt')

    # test model to get test loss and accuracy
    test_accuracy, test_loss = test_loop(test_dataloader, model, loss_fn)
    test_acc_list.append(test_accuracy)
    
print(f'Test accuracy for n_heads 4: {test_acc_list[0]:.2f}%')
print(f'Test accuracy for n_heads 8: {test_acc_list[1]:.2f}%')

C:\Users\Joey Quah\AppData\Local\Temp\ipykernel_15160\1295966561.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embed = nn.Parameter(torch.tensor(get_positional_embedding(self.n_patches ** 2 + 1, self.hidden_dim)))


Epoch 1 loss: 2.09
Validation accuracy: 36.03%


Epoch 2 loss: 1.86
Validation accuracy: 59.81%


Epoch 3 loss: 1.82
Validation accuracy: 64.09%


Epoch 4 loss: 1.80
Validation accuracy: 65.90%


...

Epoch 8 loss: 1.81
Validation accuracy: 64.98%


Epoch 9 loss: 1.81
Validation accuracy: 65.50%


Epoch 10 loss: 1.81
Validation accuracy: 65.35%


Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████| 12/12 [01:26<00:00,  7.21s/it]

Test loss: 1.82
Test accuracy: 63.57%
Test accuracy for n_heads 4: 71.03%
Test accuracy for n_heads 8: 63.57%


### Final Training (On whole dataset)

In [51]:
# load data
train_dataloader = DataLoader(training_data, shuffle=True, batch_size = batch_size)
test_dataloader = DataLoader(test_data, shuffle=True, batch_size = batch_size)

In [15]:
import time

chw = (1, 64, 64) # image dimensions
output_dim = 10 # Fashion MNIST has 10 classes
n_patches = 16

# optimal parameters
hidden_dim = 8 # number of features in each patch's representation
n_encodelayers = 8
n_heads = 4 # no of attention heads

learning_rate = 0.005
batch_size = 512
num_epochs = 20

# instantiate model
model = VisionTransformer(chw, n_patches, hidden_dim, n_encodelayers, n_heads, output_dim)

# instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train model with training data
time_taken = []
for epoch in range(num_epochs):
    start_time = time.time()
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch)
    end_time = time.time()
    print(f'Time taken for Epoch {epoch+1}: {end_time - start_time}')
    time_taken.append(end_time - start_time)
    torch.save(model, f'./final_model.pt')

# obtain training time
total_time = sum(time_taken)
print(f'Total training time = {total_time}')

# test model to get test loss and accuracy
test_accuracy, test_loss = test_loop(test_dataloader, model, loss_fn)

C:\Users\Joey Quah\AppData\Local\Temp\ipykernel_12380\1295966561.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embed = nn.Parameter(torch.tensor(get_positional_embedding(self.n_patches ** 2 + 1, self.hidden_dim)))


Epoch 1 loss: 2.19
Validation accuracy: 26.56%
Time taken for Epoch 1: 2010.0962772369385


Epoch 2 loss: 1.90
Validation accuracy: 56.31%
Time taken for Epoch 2: 1997.5132689476013


Epoch 3 loss: 1.80
Validation accuracy: 66.81%
Time taken for Epoch 3: 1809.9591801166534


Epoch 4 loss: 1.76
Validation accuracy: 70.00%
Time taken for Epoch 4: 1903.035394668579


...

Epoch 18 loss: 1.65
Validation accuracy: 81.05%
Time taken for Epoch 18: 1651.6969330310822


Epoch 19 loss: 1.65
Validation accuracy: 80.82%
Time taken for Epoch 19: 1651.7701678276062


Epoch 20 loss: 1.65
Validation accuracy: 81.41%
Time taken for Epoch 20: 1653.6248803138733
Total training time = 56107.02677965164


Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████| 20/20 [03:54<00:00, 11.71s/it]

Test loss: 1.66
Test accuracy: 80.44%


In [4]:
# obtain training time of final model

# remove 14th epoch because anomalous time taken (error due to laptop issues)
time_taken_10epochs = time_taken[:13] + time_taken[14:]

total_time_mins = sum(time_taken_10epochs) / 60
print(f'Total Training Time: {total_time_mins} minutes')

avg_time = total_time_mins/19
print(f'Average Time Per Epoch: {avg_time} minutes')

Total Training Time: 583.3759240349134 minutes
Average Time Per Epoch: 30.70399600183755 minutes
